In [1]:
## importing libraries
import pandas as pd
from datetime import datetime
from datetime import timedelta
import re

In [2]:
## scrappind data
data_prices=pd.read_html('https://www.eia.gov/dnav/ng/hist/rngwhhdD.htm')[5]
data_prices.head() 

,Week Of,Mon,Tue,Wed,Thu,Fri
0,1997 Jan- 6 to Jan-10,NaN,3.82,3.80,3.61,3.92
1,1997 Jan-13 to Jan-17,4.00,4.01,4.34,4.71,3.91
2,1997 Jan-20 to Jan-24,3.26,2.99,3.05,2.96,2.62
3,1997 Jan-27 to Jan-31,2.98,3.05,2.91,2.86,2.77
4,1997 Feb- 3 to Feb- 7,2.49,2.59,2.65,2.51,2.39


In [3]:
## data cleaning and conversion to right format
data_prices.dropna(how='all',inplace=True)
data_prices.fillna('not available',inplace=True)
#data_prices.columns = data_prices.iloc[0]
#data_prices=data_prices.iloc[1:]
data_prices.head() 

,Week Of,Mon,Tue,Wed,Thu,Fri
0,1997 Jan- 6 to Jan-10,not available,3.82,3.8,3.61,3.92
1,1997 Jan-13 to Jan-17,4,4.01,4.34,4.71,3.91
2,1997 Jan-20 to Jan-24,3.26,2.99,3.05,2.96,2.62
3,1997 Jan-27 to Jan-31,2.98,3.05,2.91,2.86,2.77
4,1997 Feb- 3 to Feb- 7,2.49,2.59,2.65,2.51,2.39


In [4]:
## getting dates in string format
data_prices['start_year']=data_prices['Week Of'].apply(lambda x:x[0:4])
data_prices['start_date']=data_prices['Week Of'].apply(lambda x:x[5:11])
data_prices['end_date']=data_prices['Week Of'].apply(lambda x:x[15:21])

In [5]:
## changing end_year as start_year+1 where end_date is in the last of dec
data_prices['year_modification']=data_prices['start_date'].apply(lambda x:x in ('Dec-31','Dec-30','Dec-29','Dec-28'))
data_prices['start_year']=data_prices['start_year'].apply(lambda x:int(x))
data_prices.loc[data_prices.year_modification == True, 'end_year'] = data_prices.loc[data_prices.year_modification == True, 'start_year']+1
data_prices.loc[data_prices.year_modification == False, 'end_year'] = data_prices.loc[data_prices.year_modification == False, 'start_year']
data_prices['end_year']=data_prices['end_year'].apply(lambda x:int(x))
data_prices['start_year']=data_prices['start_year'].apply(lambda x:str(x))
data_prices['end_year']=data_prices['end_year'].apply(lambda x:str(x))


In [6]:
## converting dates to datetime object
data_prices['start_date_object']=data_prices['start_year']+'-'+data_prices['start_date']
data_prices['end_date_object']=data_prices['end_year']+'-'+data_prices['end_date']
data_prices['start_date_object']=data_prices['start_date_object'].apply(lambda x:datetime.strptime(x,'%Y-%b-%d'))
data_prices['end_date_object']=data_prices['end_date_object'].apply(lambda x:datetime.strptime(x,'%Y-%b-%d'))

In [7]:
## generating all the dates
dates=[]
for j in data_prices.index:
    for i in range((data_prices.loc[j]['end_date_object']-data_prices.loc[j]['start_date_object']).days+1):
        dates.append(data_prices.loc[j]['start_date_object']+timedelta(days=i))

In [8]:
## creating list of prices
values=[]
for j in range(0,len(data_prices)):
    for i in range(1,6):
        values.append(data_prices.iloc[j,i])

In [9]:
price_object=pd.DataFrame(dates,values).reset_index().rename(columns={0:'Date','index':'price'})

In [10]:
price_object.to_csv('pricing_daily.csv')